In [ ]:
!nvcc --version  # correct double dash

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-13_y2r1j
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-13_y2r1j
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-x72hj19y/wheels/ef/1d/c6/f7e47f1aa1bc9d05c4120d94f90a79cf28603ef343b0dd43ff
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter


Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmphcrz9dly".


In [ ]:
code = r'''
#include <stdio.h>
#include <cuda.h>

__global__ void maxReduce(int* input) {
    __shared__ int sdata[8]; // Shared memory for storing thread-local values

    int tid = threadIdx.x;
    sdata[tid] = input[tid];  // Copy data to shared memory
    __syncthreads();  // Synchronize threads to make sure all data is copied

    // Print initial values (for debugging)
    printf("Thread %d initial value: %d\n", tid, sdata[tid]);

    // Perform reduction in shared memory
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            // Compare and store the maximum value
            sdata[tid] = max(sdata[tid], sdata[tid + s]);
            printf("Thread %d comparing %d and %d, new value: %d\n", tid, sdata[tid], sdata[tid + s], sdata[tid]);
        }
        __syncthreads();  // Synchronize again to ensure all threads update the shared memory
    }

    // Write the result back to the original input array at the first index
    if (tid == 0) {
        input[0] = sdata[0];
        printf("Thread %d writing max value: %d\n", tid, sdata[0]);
    }
}

int main() {
    const int count = 8;
    const int size = count * sizeof(int);
    int h[count] = {13, 65, 15, 14, 33, 2, 30, 8};

    int* d;
    cudaMalloc(&d, size);  // Allocate memory on the device
    cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);  // Copy data to the device

    // Launch kernel with 8 threads in a single block
    maxReduce<<<1, count>>>(d);

    int result;
    cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);  // Copy result from device to host
    printf("Largest number is: %d\n", result);  // Print the result

    cudaFree(d);  // Free the allocated memory on the device
    return 0;
}


'''

with open('max_reduce.cu', 'w') as f:
    f.write(code)

In [ ]:
!nvcc max_reduce.cu -o max_reduce

In [ ]:
!./max_reduce

Largest number is: 13
